# Vectorisation tests

Dot product is already tested in DotTest

## Distance

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections, lines, markers, path, patches
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from geometry import dot, get_metric

In [2]:
def distance(u, v, geometry="spherical"):
    '''
        Calculate distance on the manifold between two pts
        Inputs: u, v: two vectors, represented as np.arrays
        Outputs: distance, a 1-D real number
    '''
    dotprod = dot(u,v,geometry)
#    if np.abs(dotprod) > 1:
#        print("distance: {}.{} = {:.3g}".format(u, v, dotprod))

    if geometry == "spherical":
        return np.arccos(dotprod)
    elif geometry == "hyperbolic":
        return np.arccosh(-dotprod)
    elif geometry == "euclidean":
        return np.sqrt(dot(u-v, u-v, geometry))
    else:
        print("geometry = {} is not a valid option! Try 'spherical' or 'hyperbolic'".format(geometry))

In [3]:
alpha = 0.1
beta = 1.1
a = np.array([[np.sinh(alpha), np.cosh(alpha)]]).T
b = np.array([[np.sinh(beta), np.cosh(beta)]]).T
print(a)
print(b)

[[0.10016675]
 [1.00500417]]
[[1.33564747]
 [1.66851855]]


In [4]:
distance(a, b, geometry="hyperbolic")

array([[1.]])

In [5]:
c = np.array([[-1, 0],[0, 1]]).dot(a)
d = np.array([[-1, 0],[0, 1]]).dot(b)
print(c)
print(d)

[[-0.10016675]
 [ 1.00500417]]
[[-1.33564747]
 [ 1.66851855]]


In [6]:
print(distance(a, c, geometry="hyperbolic"))
print(distance(a, d, geometry="hyperbolic"))
print(distance(b, c, geometry="hyperbolic"))
print(distance(b, d, geometry="hyperbolic"))

[[0.2]]
[[1.2]]
[[1.2]]
[[2.2]]


In [7]:
ab = np.hstack([a, b])
print(ab)
cd = np.hstack([c, d])
print(cd)
distance(ab, cd, geometry="hyperbolic")

[[0.10016675 1.33564747]
 [1.00500417 1.66851855]]
[[-0.10016675 -1.33564747]
 [ 1.00500417  1.66851855]]


array([[0.2, 1.2],
       [1.2, 2.2]])

In [8]:
distance(a, cd, geometry="hyperbolic")

array([[0.2, 1.2]])

In [9]:
def frechet_diff_vectorised(p_eval, points, geometry="spherical"):
    '''
        Calculates the differential to enable a gradient descent algorithm to find 
        the Karcher/Fréchet mean of a set of points.
        Inputs:
            p_eval: Point at which to evaluate the derivative (usually a guess at 
                    the mean). (d+1)-dimensional vector, expressed in ambient space
                    coordinates.
            points: List of points which the derivative is calculate w.r.t. to. 
                    (d+1)-dimensional vector, expressed in ambient space
                    coordinates.
            geometry: string specifying which metric and distance function to use.
        Outputs:
            Derivative: (d+1)-dimensional vector, expressed in ambient space
                        coordinates.
        Note: should vectorise to remove loop over points.
    '''
    metric = get_metric(p_eval.shape[0], geometry)
#    update = np.zeros([p_eval.shape[0], 1])
    coeffs = -2.*distance(p_eval, points, geometry)
    print("numerator = ",coeffs)
    if geometry == "spherical":
        coeffs /= np.sqrt(1.-dot(p_eval, points, geometry)**2)+ 1.e-10
    elif geometry == "hyperbolic":
        coeffs /= np.sqrt(dot(p_eval, points, geometry)**2-1.)+ 1.e-10
    print("coeffs =",coeffs)
    print("points =", points)
    print("coeffs*points = ", coeffs*points)
    return np.atleast_2d(np.sum(coeffs*points, axis=1)).T


In [10]:
frechet_diff_vectorised(a, cd, geometry="hyperbolic")

numerator =  [[-0.4 -2.4]]
coeffs = [[-1.98672863 -1.58997115]]
points = [[-0.10016675 -1.33564747]
 [ 1.00500417  1.66851855]]
coeffs*points =  [[ 0.19900415  2.12364094]
 [-1.99667055 -2.65289636]]


array([[ 2.32264509],
       [-4.64956691]])

In [11]:
def frechet_diff(p_eval, points, geometry="spherical"):
    '''
        Calculates the differential to enable a gradient descent algorithm to find 
        the Karcher/Fréchet mean of a set of points.
        Inputs:
            p_eval: Point at which to evaluate the derivative (usually a guess at 
                    the mean). (d+1)-dimensional vector, expressed in ambient space
                    coordinates.
            points: List of points which the derivative is calculate w.r.t. to. 
                    (d+1)-dimensional vector, expressed in ambient space
                    coordinates.
            geometry: string specifying which metric and distance function to use.
        Outputs:
            Derivative: (d+1)-dimensional vector, expressed in ambient space
                        coordinates.
        Note: should vectorise to remove loop over points.
    '''
    metric = get_metric(p_eval.shape[0], geometry)
    update = np.zeros([p_eval.shape[0], 1])
#    print("frechet_diff: p_eval = {}, points = {}".format(p_eval, points))
    for xi in points:
        if np.array_equal(p_eval,xi):
           continue
#        print("frechet_diff: xi =", xi)
        coeff = -2.*distance(p_eval, xi, geometry)
        print("numerator =", coeff)
        if geometry == "spherical":
            coeff /= np.sqrt(1.-dot(p_eval, xi, geometry)**2)+ 1.e-10
        elif geometry == "hyperbolic":
            coeff /= np.sqrt(dot(p_eval, xi, geometry)**2-1.)
        print("frechet_diff: coeff =", coeff)
        print("coeffs*point = ",coeff*xi)
#        update += coeff*metric.dot(xi)
        update += coeff*xi
    
    print("frechet_diff: update = {}".format(update))
    return update

In [12]:
frechet_diff(a, [c,d], geometry="hyperbolic")

numerator = [[-0.4]]
frechet_diff: coeff = [[-1.98672863]]
coeffs*point =  [[ 0.19900415]
 [-1.99667055]]
numerator = [[-2.4]]
frechet_diff: coeff = [[-1.58997115]]
coeffs*point =  [[ 2.12364094]
 [-2.65289636]]
frechet_diff: update = [[ 2.32264509]
 [-4.64956691]]


array([[ 2.32264509],
       [-4.64956691]])

In [13]:
frechet_diff_vectorised(a, [c,d], geometry="hyperbolic")

numerator =  [[[-1.63320482]
  [-2.35552805]]]
coeffs = [[[-1.79387494]
  [-1.60286941]]]
points = [array([[-0.10016675],
       [ 1.00500417]]), array([[-1.33564747],
       [ 1.66851855]])]
coeffs*points =  [[[ 0.17968662]
  [-1.61089044]]

 [[ 2.39598452]
  [-2.67441736]]]


array([[-1.43120382, -0.27843283]])

In [14]:
from geometry import project_to_tangent

In [15]:
v = np.array([[1.], [1.]])
print(v)
print(a)
print(b)
print('-'*80)
print(project_to_tangent(a, v, geometry="hyperbolic"))
print(project_to_tangent(b, v, geometry="hyperbolic"))
print('-'*80)
vv = np.hstack([v, v])
print(vv)
print(ab)
print(project_to_tangent(ab, vv, geometry="hyperbolic"))

[[1.]
 [1.]]
[[0.10016675]
 [1.00500417]]
[[1.33564747]
 [1.66851855]]
--------------------------------------------------------------------------------
project_to_tangent: point_on_manifold = [[0.10016675]
 [1.00500417]], displacement = [[1.]
 [1.]], geometry = hyperbolic
[[0.90936538]
 [0.09063462]]
project_to_tangent: point_on_manifold = [[1.33564747]
 [1.66851855]], displacement = [[1.]
 [1.]], geometry = hyperbolic
[[0.55540158]
 [0.44459842]]
--------------------------------------------------------------------------------
[[1. 1.]
 [1. 1.]]
[[0.10016675 1.33564747]
 [1.00500417 1.66851855]]
project_to_tangent: point_on_manifold = [[0.10016675 1.33564747]
 [1.00500417 1.66851855]], displacement = [[1. 1.]
 [1. 1.]], geometry = hyperbolic
[[0.90936538 0.55540158]
 [0.09063462 0.44459842]]


So project_to_tangent is functions, but is incorrect for vectorised arguments. Diagonal elements seem to be correct...

In [16]:
print("a.a =", dot(a, a, geometry="hyperbolic"))
print("b.b =", dot(b, b, geometry="hyperbolic"))
print("d.d =", dot(d, d, geometry="hyperbolic"))
p = np.hstack([a, b, d])
print("[a,b,c].[a,b,c] =", dot(p, p, geometry="hyperbolic"))
print("diagonal [a,b,c].[a,b,c] =", np.diag(np.diag(dot(p, p, geometry="hyperbolic"))))
#c = np.array([[100, 200], [100, 200]])
#print(c.dot(p))

a.a = [[-1.]]
b.b = [[-1.]]
d.d = [[-1.]]
[a,b,c].[a,b,c] = [[-1.         -1.54308063 -1.81065557]
 [-1.54308063 -1.         -4.56790833]
 [-1.81065557 -4.56790833 -1.        ]]
diagonal [a,b,c].[a,b,c] = [[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]


In [17]:
def project_to_tangent_vect(point_on_manifold, displacement, geometry="hyperbolic"):
    '''
        Given a displacement, project onto tangent space defined at point_on_manifold
        Inputs: point_on_manifold, an n-D vector in embedding space
                displacement, an n-D vector of the displacement from point_on_manifold
    '''
    print("project_to_tangent: point_on_manifold = {}, displacement = {}, geometry = {}".format(
            point_on_manifold,
            displacement,
            geometry
           )
         )

    xp_dot = dot(point_on_manifold, displacement, geometry)
    xx_dot = dot(point_on_manifold, point_on_manifold, geometry)
    #xx_dot = dot(point_on_manifold, point_on_manifold, geometry)
    print(xp_dot)
    print(xx_dot)
    print(xp_dot/xx_dot)
#    if geometry in "hyperboloid":
#        xx_dot = -1. #if on hyperboloid manifold
#    return displacement - np.einsum("i,ji->ji", xp_dot/xx_dot, point_on_manifold)
    return displacement - point_on_manifold*np.diag(xp_dot/xx_dot)

In [18]:
print('-'*78)
print(project_to_tangent_vect(a, v, geometry="hyperbolic"))
print(project_to_tangent_vect(b, v, geometry="hyperbolic"))
print('-'*78)
print(project_to_tangent_vect(ab, vv, geometry="hyperbolic"))

------------------------------------------------------------------------------
project_to_tangent: point_on_manifold = [[0.10016675]
 [1.00500417]], displacement = [[1.]
 [1.]], geometry = hyperbolic
[[-0.90483742]]
[[-1.]]
[[0.90483742]]
[[0.90936538]
 [0.09063462]]
project_to_tangent: point_on_manifold = [[1.33564747]
 [1.66851855]], displacement = [[1.]
 [1.]], geometry = hyperbolic
[[-0.33287108]]
[[-1.]]
[[0.33287108]]
[[0.55540158]
 [0.44459842]]
------------------------------------------------------------------------------
project_to_tangent: point_on_manifold = [[0.10016675 1.33564747]
 [1.00500417 1.66851855]], displacement = [[1. 1.]
 [1. 1.]], geometry = hyperbolic
[[-0.90483742 -0.90483742]
 [-0.33287108 -0.33287108]]
[[-1.         -1.54308063]
 [-1.54308063 -1.        ]]
[[0.90483742 0.58638376]
 [0.21571853 0.33287108]]
[[0.90936538 0.55540158]
 [0.09063462 0.44459842]]


In [19]:
def project_to_tangent_vect_quick(point_on_manifold, displacement, geometry="hyperbolic"):
    '''
        Given a displacement, project onto tangent space defined at point_on_manifold
        Inputs: point_on_manifold, an n-D vector in embedding space
                displacement, an n-D vector of the displacement from point_on_manifold
    '''
    print("project_to_tangent: point_on_manifold = {}, displacement = {}, geometry = {}".format(
            point_on_manifold,
            displacement,
            geometry
           )
         )

    xp_dot = np.diag(dot(point_on_manifold, displacement, geometry))
    xx_dot = np.ones(xp_dot.shape)
    if geometry in "hyperbolic":
        xx_dot *= -1. #if on hyperboloid manifold
    return displacement - (xp_dot/xx_dot)*point_on_manifold

In [24]:
print('-'*78)
print(project_to_tangent_vect_quick(a, v, geometry="hyperbolic"))
print(project_to_tangent_vect_quick(b, v, geometry="hyperbolic"))
print(project_to_tangent_vect_quick(c, v, geometry="hyperbolic"))
print('-'*78)
print(project_to_tangent_vect_quick(ab, vv, geometry="hyperbolic"))
print('-'*78)
print(project_to_tangent(c, v, geometry="hyperbolic"))
abc = np.hstack([a,b,c])
vvv = np.hstack([v,v,v])
print(project_to_tangent_vect_quick(abc, vvv, geometry="hyperbolic"))





------------------------------------------------------------------------------
project_to_tangent: point_on_manifold = [[0.10016675]
 [1.00500417]], displacement = [[1.]
 [1.]], geometry = hyperbolic
[[0.90936538]
 [0.09063462]]
project_to_tangent: point_on_manifold = [[1.33564747]
 [1.66851855]], displacement = [[1.]
 [1.]], geometry = hyperbolic
[[0.55540158]
 [0.44459842]]
project_to_tangent: point_on_manifold = [[-0.10016675]
 [ 1.00500417]], displacement = [[1.]
 [1.]], geometry = hyperbolic
[[ 1.11070138]
 [-0.11070138]]
------------------------------------------------------------------------------
project_to_tangent: point_on_manifold = [[0.10016675 1.33564747]
 [1.00500417 1.66851855]], displacement = [[1. 1.]
 [1. 1.]], geometry = hyperbolic
[[0.90936538 0.55540158]
 [0.09063462 0.44459842]]
------------------------------------------------------------------------------
project_to_tangent: point_on_manifold = [[-0.10016675]
 [ 1.00500417]], displacement = [[1.]
 [1.]], geometry

In [21]:
from geometry import exponential_map

In [28]:
def exponential_map_vect(point_on_manifold, v_TpS, geometry="spherical"):
    '''
        Projects vector from tangent space of point_on_manifold onto manifold
        Inputs:
                point_on_manifold is a tf.Tensor, the initial n-D point, or 
                an array of n-D points on the manifold, 
                v_TpS is a tf.Tensor, the n-D vector, or array of such vectors,
                in tangent space
    '''
    norm_v_TpS = np.diag(np.sqrt(dot(v_TpS, v_TpS, geometry)))
       
    if geometry == "spherical":
        #if abs(norm_v_TpS.squeeze()) < 1e-8:
        #    return point_on_manifold
        #mapped_pt = tf.cond(norm_v_TpS.squeeze()) < 1e-8
        return np.cos(norm_v_TpS)*point_on_manifold + (np.sin(norm_v_TpS)/norm_v_TpS)*v_TpS
    elif geometry == "hyperbolic":
    #    print(norm_v_TpS)
    #    print(np.where(np.greater(norm_v_TpS , 0.), "Y", "N"))
        return np.where(
                        np.greater(norm_v_TpS , 0.),
                        np.cosh(norm_v_TpS)*point_on_manifold 
                            + (np.sinh(norm_v_TpS)/norm_v_TpS)*v_TpS,
                        point_on_manifold
        )
    else:
        print("geometry = {} is not a valid option! Try 'spherical' or 'hyperbolic'".format(geometry))

In [29]:
v_TaS = project_to_tangent_vect_quick(a, v, geometry="hyperbolic") 
v_TbS = project_to_tangent_vect_quick(b, v, geometry="hyperbolic")
print("Displace from points {} and {} by {} and {}".format(a, b, v_TaS, v_TbS))
print("original") 
print(exponential_map(v_TaS, a, geometry="hyperbolic"))
print(exponential_map(v_TbS, b, geometry="hyperbolic"))
print("vectorised")
print(exponential_map_vect(ab, np.hstack([v_TaS, v_TbS]), geometry="hyperbolic"))

project_to_tangent: point_on_manifold = [[0.10016675]
 [1.00500417]], displacement = [[1.]
 [1.]], geometry = hyperbolic
project_to_tangent: point_on_manifold = [[1.33564747]
 [1.66851855]], displacement = [[1.]
 [1.]], geometry = hyperbolic
Displace from points [[0.10016675]
 [1.00500417]] and [[1.33564747]
 [1.66851855]] by [[0.90936538]
 [0.09063462]] and [[0.55540158]
 [0.44459842]]
original
[[1.1826795 ]
 [1.54878365]]
[[1.9760455 ]
 [2.21466833]]
vectorised
[[1.1826795  1.9760455 ]
 [1.54878365 2.21466833]]


In [31]:
print(ab)

[[0.10016675 1.33564747]
 [1.00500417 1.66851855]]


In [37]:
np.stack([a,b], axis=1).squeeze()

array([[0.10016675, 1.33564747],
       [1.00500417, 1.66851855]])

In [38]:
np.tile(a, 2)

array([[0.10016675, 0.10016675],
       [1.00500417, 1.00500417]])